In [ ]:
pip install minsearch

In [ ]:
import minsearch


In [ ]:
import json

In [ ]:
with open("documents.json", "rt") as f_in:
    docs_raw = json.load(f_in)

In [ ]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict["documents"]:
        doc["course"] = course_dict["course"]
        documents.append(doc)

In [ ]:
documents[0]

In [ ]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [ ]:
index.fit(documents)

In [ ]:
q = "the course has already started, can I still enroll?"

In [ ]:
boost = {"question": 3.0, "section": 0.5} #default is 1

results = index.search(
    query=q,
    boost_dict=boost,
    num_results=5,
    filter_dict={"course": "data-engineering-zoomcamp"}
)

In [ ]:
results

In [ ]:
from groq import Groq

In [ ]:
client = Groq()

In [ ]:
q

In [ ]:
response = client.chat.completions.create(
    model="compound-beta",
    messages=[{"role": "user", "content": q}]
)

In [ ]:
response.choices[0].message

In [ ]:
prompt_template = """
You're a course teaching assistant. Anser the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: {question}

CONTEXT: {context}
""".strip()

context = ""

for doc in results:
    context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [ ]:
context = ""

for doc in results:
    context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [ ]:
print(context)

In [ ]:
prompt = prompt_template.format(question=q, context=context).strip()

In [ ]:
print(prompt)

In [ ]:
response = client.chat.completions.create(
    model="compound-beta",
    messages=[{"role": "user", "content": prompt}]
)

response.choices[0].message.content

In [66]:
def search(query):
    boost = {"question": 3.0, "section": 0.5}
    results = index.search(
        query=query,
        boost_dict=boost,
        num_results=10,
        filter_dict={"course": "data-engineering-zoomcamp"}
    )

    return results

In [67]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Anser the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: {question}

CONTEXT: {context}
""".strip()

    context = ""

    for doc in search_results:
        context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt
        

In [68]:
def llm(prompt):
    response = client.chat.completions.create(
    model="compound-beta",
    messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

In [70]:
query = "how do I run kafka"



In [71]:
rag(query)

'To answer the question "how do I run kafka," I will search the CONTEXT for any information related to running Kafka.\n\nThe CONTEXT provided contains several sections related to Kafka, but none of them directly answer the question "how do I run kafka." However, one section provides information on how to run a Kafka producer:\n\nsection: Module 6: streaming with kafka\nquestion: Java Kafka: How to run producer/consumer/kstreams/etc in terminal\nanswer: In the project directory, run:\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n\nAnother section provides information on how to fix an error related to Kafka brokers:\n\nsection: Module 6: streaming with kafka\nquestion: kafka.errors.NoBrokersAvailable: NoBrokersAvailable\nanswer: If you have this error, it most likely that your kafka broker docker container is not working.\nUse docker ps to confirm\nThen in the docker compose yaml file folder, run docker compose up -d to start all the in

In [72]:
rag("the course has already started, can I still enroll?")

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

In [73]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [121]:
from elasticsearch import Elasticsearch

In [122]:
ec_client = Elasticsearch("http://localhost:9200")

In [123]:
ec_client.info()

ObjectApiResponse({'name': '089eef0efac9', 'cluster_name': 'docker-cluster', 'cluster_uuid': '2qzVOdt9TSuVMYeHplDoWA', 'version': {'number': '9.0.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '0a58bc1dc7a4ae5412db66624aab968370bd44ce', 'build_date': '2025-05-28T10:06:37.834829258Z', 'build_snapshot': False, 'lucene_version': '10.1.0', 'minimum_wire_compatibility_version': '8.18.0', 'minimum_index_compatibility_version': '8.0.0'}, 'tagline': 'You Know, for Search'})

In [124]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

ec_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [125]:
from tqdm.auto import tqdm

In [126]:
for doc in tqdm(documents):
    ec_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [132]:
query = "I just discovered the course. Can I still join?"

In [137]:
def elastic_search(query):
    
    search_query = {
    "size": 5, #number of results 
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^3", "text", "section"],  # ^ means boost
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}
    response = ec_client.search(index=index_name, body=search_query)
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    return result_docs

In [139]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [140]:
rag(query)

'Based on the CONTEXT provided, the answer to the QUESTION "I just discovered the course. Can I still join?" is:\n\nYes, even if you don\'t register, you\'re still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don\'t leave everything for the last minute.\n\nThis answer is derived from the CONTEXT, specifically from the question "Course - Can I still join the course after the start date?" which implies that joining the course after discovering it is possible. Additionally, the answer to the question "Course - I have registered for the Data Engineering Bootcamp. When can I expect to receive the confirmation email?" mentions that registration is not required to start learning and submitting homework, which further supports the idea that joining the course is possible even after discovering it.'